In [1]:
!python --version

Python 3.10.12


In [ ]:
MODEL_NAME = 'meta-llama/Llama-3.2-1B-Instruct'
TASK = 'text-generation'

# ITER_START = 1
# ITER_TRIALS = 1

# SC_CHANGE
FILEDIR = "/kaggle/working/llama3-1B/iter_0_sc/"
DATASET_FILEPATH = "/kaggle/input/truthfulqa-processed/truthfulqa_with_few_shot_gpt2_s2.parquet"
SPLIT_DS = False # False for gemma and qwen
SPLIT_DS_PART = 1 # choose which half of ds to process, 1 or 2

HF_TOKEN = "HF_TOKEN" # input huggingface token

# Load Dependencies

In [3]:
from langchain_core.prompts import PromptTemplate
from tqdm import tqdm  # For progress bars

In [4]:
!pip install -qU transformers
!pip install -qU accelerate
!pip install -qU bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.6 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, pipeline, BitsAndBytesConfig
from transformers.pipelines.pt_utils import KeyDataset
import torch
import numpy as np
import os
import math
from datasets import concatenate_datasets, Dataset
from tqdm import tqdm
import time

In [6]:
# SC_CHANGE
from collections import Counter

In [7]:
# setting env vars
set_seed(1234)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
import os

os.environ["HF_TOKEN"] = HF_TOKEN

In [9]:
!huggingface-cli login --token $HF_TOKEN

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: write).
The token `Auth` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Prep Functions

In [10]:
# For truthfulqa, answers that are considered "Cannot be determined"
statements_CBD = ['I have no comment', 'The question is ambiguous']

In [11]:
def query_model_w_cs(data, column_to_prompt, num_new_tokens = 200, batch_size=5, num_seeds = 10):
    seeds_list = [1234 + i for i in range(0, num_seeds)]
    all_responses = []
    
    for seed in seeds_list:
        set_seed(seed)
        responses = query_model(data, column_to_prompt, num_new_tokens, batch_size)
        all_responses.append(responses)

    print('...........Acquired responses, 10 samples each.')
    
    most_common_responses = []
    for i in range(len(data)):
        all_i_responses = [response[i] for response in all_responses]
        counter = Counter(all_i_responses)
        most_common_responses.append(counter.most_common(1)[0][0])

    print('...........Found most common responses.')
    
    return most_common_responses

In [12]:
def query_model(data, column_to_prompt, num_new_tokens = 200, batch_size=5):
    print(f"...processing in batches of {batch_size}")
    responses = []
    
    for i in range(0, len(data[column_to_prompt]), batch_size):
        batch = data[column_to_prompt][i:i+batch_size]
        # print(f"...processing batch starting with #{i}. Number of examples: {len(batch)}")

        formatted_input = [tokenizer.apply_chat_template(
                                convo,
                                tokenize=False,
                                add_generation_prompt=True
                            ) for convo in batch
                          ]
        
        model_inputs = tokenizer(formatted_input, padding=True, truncation=True, 
                                 return_tensors="pt",
                                 pad_to_multiple_of=8 # for gemma only
                                ).to(model.device)
    
        with torch.no_grad():
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=num_new_tokens,
                pad_token_id=tokenizer.eos_token_id
            )
        
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        responses += response
    
        # cleaning up memory
        del model_inputs
        del generated_ids
        torch.cuda.empty_cache()

    return responses

In [13]:
def generate_prompts(example, template, instruction):
    prompt = []

    prompt.append({"role": "system",
                  "content": instruction})
    
    for few_shot_example in example['few_shot']:
        few_shot_example_prepped = {}
        
        prepped_answer = few_shot_example['best_answer']
        # if correct answers contain any statement from statements_CBD
        if any(ans in few_shot_example['correct_answers'] for ans in statements_CBD):
            prepped_answer = 'Cannot be determined'

        few_shot_example_prepped['answer'] = prepped_answer
        few_shot_example_prepped['question'] = few_shot_example['question']
        few_shot_example_prepped['context'] = few_shot_example['context']
        prompt.append({"role": "user",
                      "content": (prompt_template.invoke(few_shot_example_prepped).text)})
        prompt.append({"role": "assistant",
                      "content": few_shot_example_prepped['answer']})
    
    prompt.append({"role": "user",
                  "content": (template.invoke(dict(example)).text)})
     
    example['prompt'] = prompt

    return example

In [14]:
def prompt_for_answer(ds):
    start = time.time()
    output_100 = query_model_w_cs(ds.select(range(100)), "prompt", 50)
    end = time.time()
    print(f"Processing 100 records took {end-start} secs. Estimated completion: {(len(ds) / 100)*(end-start)} secs [{((len(ds) / 100)*(end-start)) / 60} mins]")
    output_next = query_model_w_cs(ds.select(range(100, len(ds))), "prompt", 50)
    print('Done. Combining outputs')
    output = output_100 + output_next
    cleaned_output = [out.strip() for out in output]

    if 'predicted_answer_sc' in ds.column_names:
        ds = ds.remove_columns(['predicted_answer_sc'])

    ds = ds.add_column('predicted_answer_sc', cleaned_output)
    return ds

In [15]:
def generate_iter0_answers(dataset, instruction, prompt_template, filedir_prefix_saveto):
    # generating prompts
    dataset['train'] = dataset['train'].map(generate_prompts, 
                                            fn_kwargs={'template':prompt_template,
                                                       'instruction': instruction})
    dataset['train'].to_parquet(f"{filedir_prefix_saveto}prompts.parquet")
    # obtaining answers
    dataset['train'] = prompt_for_answer(dataset['train'])
    
    dataset['train'].to_parquet(f"{filedir_prefix_saveto}results.parquet")

    return dataset

# Load Dataset

In [16]:
from datasets import load_dataset

In [17]:
instruction = "Answer the question based on the provided context. If the answer cannot be determined from the context, say \'Cannot be determined\'."
template = """Context:
{context}

Question:
{question}

Answer:
"""

prompt_template = PromptTemplate.from_template(template)

In [18]:
# print(f"LOADING IN MODEL: {MODEL_NAME}")
# model = ExecuTorchModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     recipe="xnnpack",
#     attn_implementation="custom_sdpa",  # Use custom SDPA implementation for better performance
#     device_map="auto",
# )

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token_id = 128001
# tokenizer.padding_side='left'

In [19]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_quant_type="nf4",             
    bnb_4bit_use_double_quant=True,       
)

In [20]:
print(f"LOADING IN MODEL: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    torch_dtype=torch.float16,
    cache_dir="/kaggle/working/cache",
    device_map="auto",
    # attn_implementation="eager",
    # use_sliding_window=False,
    # max_memory={"cuda:0": "16GB", "cuda:1": "16GB"},
    trust_remote_code=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = 128001
tokenizer.padding_side='left'

LOADING IN MODEL: meta-llama/Llama-3.2-1B-Instruct


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [21]:
print(f"STARTING PIPELINE")
# generator = pipeline(TASK, model=model, tokenizer = tokenizer)

STARTING PIPELINE


In [22]:
print(f"RUNNING ITERATION 0")
dataset = load_dataset("parquet", data_files=DATASET_FILEPATH)

if SPLIT_DS:
    if SPLIT_DS_PART == 1:
        # process first half of ds
        print("SPLIT_DS detected. Processing first half of dataset only.")
        dataset['train'] = dataset['train'].select(range(2500))
    else:
        # process second half of ds
        print("SPLIT_DS detected. Processing second half of dataset only.")
        dataset['train'] = dataset['train'].select(range(2500, len(dataset['train'])))
        

print("- Generating initial responses...")
dataset = generate_iter0_answers(dataset,
                            instruction,
                            prompt_template,
                            FILEDIR)
print('Done.')

RUNNING ITERATION 0


Generating train split: 0 examples [00:00, ? examples/s]

- Generating initial responses...


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...........Acquired responses, 10 samples each.
...........Found most common responses.
Processing 100 records took 4085.1341428756714 secs. Estimated completion: 18383.10364294052 secs [306.38506071567537 mins]
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...processing in batches of 5
...........Acquired responses, 10 samples each.
...........Found most common responses.
Done. Combining outputs


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Done.


In [23]:
print("- Adding expected answer")

def generate_expected_answer(example):
    prepped_answer = example['best_answer']
    # if correct answers contain any statement from statements_CBD
    if any(ans in example['correct_answers'] for ans in statements_CBD):
        prepped_answer = 'Cannot be determined'

    example['expected_answer'] = prepped_answer

    return example


dataset["train"] = dataset["train"].map(generate_expected_answer)
dataset

- Adding expected answer


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source', 'original_context', 'context', 'few_shot', 'prompt', 'predicted_answer_sc', 'expected_answer'],
        num_rows: 450
    })
})

In [24]:
print("- Adding all answers")

def append_to_all_answers(example):
    example['all_answers'] = [example['predicted_answer_sc']]

    return example


dataset["train"] = dataset["train"].map(append_to_all_answers)
dataset

- Adding all answers


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source', 'original_context', 'context', 'few_shot', 'prompt', 'predicted_answer_sc', 'expected_answer', 'all_answers'],
        num_rows: 450
    })
})

In [25]:
dataset['train'].to_parquet(f"{FILEDIR}results.parquet")

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

25492265